{/* cspell:ignore nfev */}

# Variational quantum eigensolver
*Usage estimate: 73 minutes on IBM Kyiv (NOTE: This is an estimate only. Your runtime may vary.)*

## Background

[Variational quantum algorithms](https://arxiv.org/abs/2012.09265) are promising candidate hybrid-algorithms for observing quantum computation utility on noisy near-term devices. Variational algorithms are characterized by the use of a classical optimization algorithm to iteratively update a parameterized trial solution, or "ansatz". Chief among these methods is the Variational Quantum Eigensolver (VQE) that aims to solve for the ground state of a given Hamiltonian represented as a linear combination of Pauli terms, with an ansatz circuit where the number of parameters to optimize over is polynomial in the number of qubits.  Given that the size of the full solution vector is exponential in the number of qubits, successful minimization using VQE requires, in general, additional problem-specific information to define the structure of the ansatz circuit.


Executing a VQE algorithm requires the following components:

   1. Hamiltonian and ansatz (problem specification)
   2. Qiskit Runtime estimator
   3. Classical optimizer

Although the Hamiltonian and ansatz require domain-specific knowledge to construct, these details are immaterial to the Runtime, and we can execute a wide class of VQE problems in the same manner.

## Requirements

Before starting this tutorial, ensure that you have the following installed:

- Qiskit SDK 1.0 or later, with visualization support (`pip install 'qiskit[visualization]'`)
- Qiskit Runtime 0.22 or later(`pip install qiskit-ibm-runtime`)

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

from qiskit.circuit.library import efficient_su2
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import EstimatorV2 as Estimator

## Step 1: Map classical inputs to a quantum problem

Although the problem instance in question for the VQE algorithm can come from a variety of domains, the form for execution through Qiskit Runtime is the same. Qiskit provides a convenience class for expressing Hamiltonians in Pauli form, and a collection of widely used ansatz circuits in the [`qiskit.circuit.library`](https://docs.quantum-computing.ibm.com/api/qiskit/circuit_library).

This example Hamiltonian is derived from a quantum chemistry problem.

In [ ]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)

In [3]:
hamiltonian = SparsePauliOp.from_list(
    [("YZ", 0.3980), ("ZI", -0.3980), ("ZZ", -0.0113), ("XX", 0.1810)]
)

Our choice of ansatz is the `efficient_su2` that, by default, linearly entangles qubits, making it ideal for quantum hardware with limited connectivity.

In [ ]:
ansatz = efficient_su2(hamiltonian.num_qubits)
ansatz.decompose().draw("mpl", style="iqp")

<Image src="/docs/images/tutorials/variational-quantum-eigensolver/extracted-outputs/59bffe5e-0.avif" alt="Output of the previous code cell" />

From the previous figure we see that our ansatz circuit is defined by a vector of parameters, $\theta_{i}$, with the total number given by:

In [5]:
num_params = ansatz.num_parameters
num_params

16

## Step 2: Optimize problem for quantum hardware execution

To reduce the total job execution time, Qiskit primitives only accept circuits (ansatz) and observables (Hamiltonian) that conform to the instructions and connectivity supported by the target QPU (referred to as instruction set architecture (ISA) circuits and observables).

### ISA circuit

Schedule a series of [`qiskit.transpiler`](https://docs.quantum-computing.ibm.com/api/qiskit/transpiler) passes to optimize the circuit for a selected backend and make it compatible with the backend's ISA. This can be easily done with a preset pass manager from `qiskit.transpiler` and its `optimization_level` parameter.

The lowest optimization level does the minimum needed to get the circuit running on the device; it maps the circuit qubits to the device qubits and adds swap gates to allow all two-qubit operations. The highest optimization level is much smarter and uses lots of tricks to reduce the overall gate count. Since multi-qubit gates have high error rates and qubits decohere over time, the shorter circuits should give better results.

In [ ]:
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

ansatz_isa = pm.run(ansatz)

In [7]:
ansatz_isa.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="/docs/images/tutorials/variational-quantum-eigensolver/extracted-outputs/20d9923c-0.avif" alt="Output of the previous code cell" />

### ISA observable

Transform the Hamiltonian to make it backend-compatible before running jobs with [`Runtime Estimator V2`](/docs/api/qiskit-ibm-runtime/estimator-v2#run). Perform the transformation by using the `apply_layout` method of `SparsePauliOp` object.

In [8]:
hamiltonian_isa = hamiltonian.apply_layout(layout=ansatz_isa.layout)

## Step 3: Execute using Qiskit primitives

Like many classical optimization problems, the solution to a VQE problem can be formulated as minimization of a scalar cost function.  By definition, VQE looks to find the ground state solution to a Hamiltonian by optimizing the ansatz circuit parameters to minimize the expectation value (energy) of the Hamiltonian.  With the Qiskit Runtime [`Estimator`](/docs/api/qiskit-ibm-runtime/estimator-v2) directly taking a Hamiltonian and parameterized ansatz, and returning the necessary energy, the cost function for a VQE instance is quite simple.

Note that the `run()` method of [Qiskit Runtime `EstimatorV2`](/docs/api/qiskit-ibm-runtime/estimator-v2)  takes an iterable of `primitive unified blocs (PUBs)`. Each PUB is an iterable in the format `(circuit, observables, parameter_values: Optional, precision: Optional)`.

In [9]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance
        cost_history_dict: Dictionary for storing intermediate results

    Returns:
        float: Energy estimate
    """
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    energy = result[0].data.evs[0]

    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(energy)
    print(
        f"Iters. done: {cost_history_dict['iters']} [Current cost: {energy}]"
    )

    return energy

Note that, in addition to the array of optimization parameters that must be the first argument, we use additional arguments to pass the terms needed in the cost function. We also access a global variable called `cost_history_dict` within this function. This dictionary stores the current vector at each iteration, for example in case you need to restart the routine due to failure, and also returns the current iteration number and average time per iteration.

In [10]:
cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

We can now use a classical optimizer of our choice to minimize the cost function. Here, we use the [COBYLA routine from SciPy through the `minimize` function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html). Note that when running on real quantum hardware, the choice of optimizer is important, as not all optimizers handle noisy cost function landscapes equally well.

To begin the routine, specify a random initial set of parameters:

In [11]:
x0 = 2 * np.pi * np.random.random(num_params)

In [12]:
x0

array([2.51632747, 0.41779892, 5.85800259, 4.83749838, 3.20828874,
       5.23058321, 0.23909191, 5.93347588, 0.98307886, 5.8564212 ,
       3.41519817, 2.07444879, 0.7790487 , 0.72421971, 1.09848722,
       3.31663941])

Because we are sending a large number of jobs that we would like to execute iteratively, we use a [`Session`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.Session) to execute all the generated circuits in one block.  Here `args` is the standard SciPy way to supply the additional parameters needed by the cost function.

In [13]:
with Session(backend=backend) as session:
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 10000

    res = minimize(
        cost_func,
        x0,
        args=(ansatz_isa, hamiltonian_isa, estimator),
        method="cobyla",
    )

Iters. done: 1 [Current cost: -0.41297466987766457]
Iters. done: 2 [Current cost: -0.4074417945791059]
Iters. done: 3 [Current cost: -0.3547606824167446]
Iters. done: 4 [Current cost: -0.28168907720107683]
Iters. done: 5 [Current cost: -0.4578159863650208]
Iters. done: 6 [Current cost: -0.3086594190935841]
Iters. done: 7 [Current cost: -0.15483382641187454]
Iters. done: 8 [Current cost: -0.5349854031388536]
Iters. done: 9 [Current cost: -0.3895567829431972]
Iters. done: 10 [Current cost: -0.4820250378135054]
Iters. done: 11 [Current cost: -0.18395917730188865]
Iters. done: 12 [Current cost: -0.5311791591751455]
Iters. done: 13 [Current cost: -0.49602925858832053]
Iters. done: 14 [Current cost: -0.3482916173440449]
Iters. done: 15 [Current cost: -0.5668230348151325]
Iters. done: 16 [Current cost: -0.521882967057639]
Iters. done: 17 [Current cost: -0.26976382760304113]
Iters. done: 18 [Current cost: -0.051127606468041646]
Iters. done: 19 [Current cost: -0.5213490424900524]
Iters. done: 2

At the terminus of this routine we have a result in the standard SciPy `OptimizeResult` format.  From this we see that it took `nfev` number of cost function evaluations to obtain the solution vector of parameter angles (`x`) that, when plugged into the ansatz circuit, yield the approximate ground state solution we were looking for.

In [14]:
res

 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -0.634701203143904
       x: [ 2.581e+00  4.153e-01 ...  1.070e+00  3.123e+00]
    nfev: 146
   maxcv: 0.0

## Step 4: Post-process and return result in desired classical format

If the procedure terminates correctly, then the `prev_vector` and `iters` values in our `cost_history_dict` dictionary should be equal to the solution vector and total number of function evaluations, respectively.  This is easy to verify:

In [15]:
all(cost_history_dict["prev_vector"] == res.x)

True

In [16]:
cost_history_dict["iters"] == res.nfev

False

We can also now view the progress toward convergence as monitored by the cost history at each iteration:

In [17]:
fig, ax = plt.subplots()
ax.plot(range(cost_history_dict["iters"]), cost_history_dict["cost_history"])
ax.set_xlabel("Iterations")
ax.set_ylabel("Cost")
plt.draw()

<Image src="/docs/images/tutorials/variational-quantum-eigensolver/extracted-outputs/8501d609-0.avif" alt="Output of the previous code cell" />

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_9AizRyKFHEGzVs2)